## Eigene Umgebung

Wieder ein paar Begriffe:

Spaces:
- Box
    - Bereich zwischen 2 Zahlen
 
- Discrete
    - Liste mit Werten von 0 bis Maximum
 
- Tuple
    - Kombiniert mehrere Spaces zusammen
 
- Dict
    - Benanntes Tupel

In [26]:
import gymnasium as gym
import numpy as np
import gymnasium.spaces as sp
from typing import Optional

In [7]:
disc = sp.Discrete(3)

In [8]:
for x in range(100):
    print(disc.sample())

2
1
1
1
0
2
0
2
2
1
0
2
0
2
1
2
0
1
2
1
2
2
1
2
1
1
2
1
0
1
1
1
1
1
0
2
1
0
0
2
2
0
0
1
0
1
2
1
1
1
1
0
0
0
1
2
0
1
0
1
1
2
0
1
1
1
1
1
0
2
0
0
0
2
2
0
0
0
1
0
1
1
1
2
1
2
1
0
1
2
2
1
1
1
1
1
1
0
2
1


In [10]:
box = sp.Box(0, 100)

In [11]:
box.sample()

array([86.630585], dtype=float32)

In [12]:
boxShape = sp.Box(0, 100, shape=(3, 3))

In [15]:
boxShape.sample()

array([[26.525356, 34.17661 , 11.240754],
       [60.992878, 16.596964, 45.657505],
       [67.25015 , 85.00008 , 24.594597]], dtype=float32)

In [18]:
sp.Tuple((sp.Discrete(3), sp.Discrete(3)))

Tuple(Discrete(3), Discrete(3))

In [22]:
sp.Dict({"X": sp.Box(0, 10), "Y": sp.Box(0, 5)})

OrderedDict([('X', array([5.20383], dtype=float32)),
             ('Y', array([2.9084616], dtype=float32))])

## Eigene Umgebung

Muss eine Klasse sein die von gym.Env erbt

Dadurch müssen die Funktionen step() und reset() implementiert werden

- Observation Space: Die Daten die beobachtet werden (der State nach einem Step)
- Action Space: Die möglichen Aktionen des Agenten (z.B. Links, Rechts)
- State: Der derzeitige Status

In [95]:
class CarDriver(gym.Env):
    def __init__(self):
        self.action_space = sp.Discrete(3)  # Die verschiedenen Aktionen: Bremsen, Beschleunigen, Nix tun
        self.observation_space = sp.Box(0, 100, dtype=np.int8)  # Die Geschwindigkeit zwischen 0 und 100
        self.state = 40 + np.random.randint(0, 10)  # Startgeschwindigkeit
        self.drive_length = 60  # Länge der Fahrt
        
    def step(self, action):
        # Action verwendet
        # 0: Nix (+0km/h)
        # 1: Bremsen (-3km/h)
        # 2: Beschleunigen (+3km/h)
        if action == 1:
            self.state -= 3
        if action == 2:
            self.state += 3

        # Belohnung berechnen
        # Zwischen 45 und 55 km/h -> +1 Punkt sonst -1 Punkt
        if self.state >= 45 and self.state <= 55:
            reward = 1
        else:
            reward = -1

        # Noise
        self.state += np.random.randint(-1, 1)

        self.drive_length -= 1
        
        # State muss bei return immer ein Numpy Array sein
        return np.array([self.state], dtype=np.int8), reward, self.drive_length == 0, False, {}

    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None,):
        self.state = 40 + np.random.randint(0, 10)  # Startgeschwindigkeit
        self.drive_length = 60  # Länge der Fahrt
        return (np.array([self.state], dtype=np.int8), {})

In [101]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO

In [97]:
env = CarDriver()

In [98]:
check_env(env, warn=True)

In [99]:
durchgaenge = 5
for x in range(durchgaenge):
    state = env.reset()  # Gibt den Startstate zurück
    score = 0  # Score sollte 200 erreichen

    while True:
        # env.render()
        action = env.action_space.sample()  # Random Action auswählen
        new_state, reward, done, term, info = env.step(action)  # Action an die Umgebung weitergeben
        score += reward
        if done or term:
            break

    print(f"Durchgang {x}: Score: {score}")

Durchgang 0: Score: -60
Durchgang 1: Score: -56
Durchgang 2: Score: -38
Durchgang 3: Score: -54
Durchgang 4: Score: -60


In [102]:
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [108]:
model.learn(total_timesteps=100000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 23       |
| time/              |          |
|    fps             | 876      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 25.6        |
| time/                   |             |
|    fps                  | 525         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008383241 |
|    clip_fraction        | 0.158       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.03       |
|    explained_variance   | 0.0243      |
|    learning_rate        | 0.

In [109]:
model.save("CarDriver.zip")

In [110]:
env = CarDriver()

In [111]:
model.load("CarDriver.zip", env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [114]:
durchgaenge = 20
for x in range(durchgaenge):
    state = env.reset()  # Gibt den Startstate zurück
    score = 0  # Score sollte 60 erreichen

    while True:
        # env.render()
        if len(state) == 2:
            action = model.predict(state[0])  # Random Action auswählen
        else:
            action = model.predict(state)
        state, reward, done, term, info = env.step(action[0])  # Action an die Umgebung weitergeben
        score += reward
        if done or term:
            break

    print(f"Barl Kenz {x}: Score: {score}")

Barl Kenz 0: Score: 60
Barl Kenz 1: Score: 54
Barl Kenz 2: Score: 60
Barl Kenz 3: Score: 58
Barl Kenz 4: Score: 56
Barl Kenz 5: Score: 52
Barl Kenz 6: Score: 48
Barl Kenz 7: Score: 56
Barl Kenz 8: Score: 58
Barl Kenz 9: Score: 54
Barl Kenz 10: Score: 54
Barl Kenz 11: Score: 60
Barl Kenz 12: Score: 60
Barl Kenz 13: Score: 60
Barl Kenz 14: Score: 54
Barl Kenz 15: Score: 60
Barl Kenz 16: Score: 60
Barl Kenz 17: Score: 54
Barl Kenz 18: Score: 52
Barl Kenz 19: Score: 58
